In [1]:
latest_gameweek = 11

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 300)
#pd.set_option('display.max_rows', 100)

# Data

In [3]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

,name,element_type,home,corners_and_indirect_freekicks_order,creativity_rank,direct_freekicks_order,ict_index_rank,influence_rank,minutes,now_cost,penalties_order,points_per_game,selected_by_percent,threat_rank,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,team_name,opponent_team,date,gameweek,expected_points
0,Aaron Cresswell,2,1,NaN,305,NaN,383,387,84,42,NaN,2.0,0.3,512,1.376051,1.384059,1.359881,1.343581,1.998102,1.90523,1.728895,1.570028,0.981528,1.015687,1.029512,1.036002,1.660111,1.642538,1.651300,1.652891,0.160368,26.701862,25.204241,3.021589,0.044699,1.999045,0.0,2.836277,0.016683,0.106816,1.318517,82.59625,3.953274,0.212206,24.623461,25.049863,3.436088,0.057114,1.741759,0.0,3.487163,0.021363,0.114283,1.342481,84.390376,3.834886,0.250714,23.742556,25.036339,3.728641,0.056075,1.61316,0.0,3.547421,0.022819,0.125008,1.34266,85.578484,3.833408,0.27402,23.455157,25.11918,3.9016,0.053568,1.55405,0.0,3.4974,0.023191,0.132523,1.337751,86.186263,3.855249,6.0,466.0,505.6,90.0,1.0,30.0,0.0,68.0,0.5128,3.10947,29.2366,1909.0,77.858508,0.282871,21.969618,23.836564,4.243059,0.047145,1.414353,0.0,3.205867,0.024176,0.146596,1.378362,3.670647,1.950078,West Ham,Nottingham Forest,2023-11-12T14:00:00Z,12,3.278376
1,Aaron Cresswell,2,0,NaN,305,NaN,383,387,84,42,NaN,2.0,0.3,512,1.376051,1.384059,1.359881,1.343581,1.998102,1.90523,1.728895,1.570028,0.953317,0.985976,1.005636,1.014291,1.741864,1.718175,1.647562,1.596921,0.160368,26.701862,25.204241,3.021589,0.044699,1.999045,0.0,2.836277,0.016683,0.106816,1.318517,82.59625,3.953274,0.212206,24.623461,25.049863,3.436088,0.057114,1.741759,0.0,3.487163,0.021363,0.114283,1.342481,84.390376,3.834886,0.250714,23.742556,25.036339,3.728641,0.056075,1.61316,0.0,3.547421,0.022819,0.125008,1.34266,85.578484,3.833408,0.27402,23.455157,25.11918,3.9016,0.053568,1.55405,0.0,3.49

(4581, 113)

In [4]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,element_type,event_points,first_name,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,influence,influence_rank,influence_rank_type,minutes,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,points_per_game,red_cards,saves,second_name,selected_by_percent,threat,threat_rank,threat_rank_type,total_points,web_name,yellow_cards,team_name,gameweek,season,gameweek_xG,gameweek_xA,gameweek_xGA,gameweek_minutes,team_xG,team_xGA,team_xG_ewm_5,team_xG_ewm_10,team_xG_ewm_20,team_xG_ewm_40,team_xGA_ewm_5,team_xGA_ewm_10,team_xGA_ewm_20,team_xGA_ewm_40,opponent_xG,opponent_xGA,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,opponent_xGA_ewm_40,home,gameweek_assists,gameweek_bps,gameweek_creativity,gameweek_goals_scored,gameweek_goals_conceded,gameweek_own_goals,gameweek_penalties_saved,gameweek_red_cards,gameweek_saves,gameweek_threat,gameweek_yellow_cards,gameweek_xPoints,gameweek_assists_ewm_5,gameweek_bps_ewm_5,gameweek_creativity_ewm_5,event_points_ewm_5,gameweek_goals_scored_ewm_5,gameweek_goals_conceded_ewm_5,gameweek_saves_ewm_5,gameweek_threat_ewm_5,gameweek_xG_ewm_5,gameweek_xA_ewm_5,gameweek_xGA_ewm_5,gameweek_minutes_ewm_5,gameweek_xPoints_ewm_5,gameweek_assists_ewm_10,gameweek_bps_ewm_10,gameweek_creativity_ewm_10,event_points_ewm_10,gameweek_goals_scored_ewm_10,gameweek_goals_conceded_ewm_10,gameweek_saves_ewm_10,gameweek_threat_ewm_10,gameweek_xG_ewm_10,gameweek_xA_ewm_10,gameweek_xGA_ewm_10,gameweek_minutes_ewm_10,gameweek_xPoints_ewm_10,gameweek_assists_ewm_20,gameweek_bps_ewm_20,gameweek_creativity_ewm_20,event_points_ewm_20,gameweek_goals_scored_ewm_20,gameweek_goals_conceded_ewm_20,gameweek_saves_ewm_20,gameweek_threat_ewm_20,gameweek_xG_ewm_20,gameweek_xA_ewm_20,gameweek_xGA_ewm_20,gameweek_minutes_ewm_20,gameweek_xPoints_ewm_20,gameweek_assists_ewm_40,gameweek_bps_ewm_40,gameweek_creativity_ewm_40,event_points_ewm_40,gameweek_goals_scored_ewm_40,gameweek_goals_conceded_ewm_40,gameweek_saves_ewm_40,gameweek_threat_ewm_40,gameweek_xG_ewm_40,gameweek_xA_ewm_40,gameweek_xGA_ewm_40,gameweek_minutes_ewm_40,gameweek_xPoints_ewm_40,gameweek_assists_expanding,gameweek_bps_expanding,gameweek_creativity_expanding,event_points_expanding,gameweek_goals_scored_expanding,gameweek_goals_conceded_expanding,gameweek_saves_expanding,gameweek_threat_expanding,gameweek_xG_expanding,gameweek_xA_expanding,gameweek_xGA_expanding,gameweek_minutes_expanding,gameweek_xPoints_expanding,gameweek_assists_expanding_per90,gameweek_bps_expanding_per90,gameweek_creativity_expanding_per90,event_points_expanding_per90,gameweek_goals_scored_expanding_per90,gameweek_goals_conceded_expanding_per90,gameweek_saves_expanding_per90,gameweek_threat_expanding_per90,gameweek_xG_expanding_per90,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_minutes_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,ep_next,ep_this,form,id,in_dreamteam,news,news_added,photo,special,squad_number,status,team,team_code,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,corners_and_indirect_freekicks_text,direct_freekicks_text,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,data_retrieved_datetime
18839,0,0,2,0,NaN,0.0,554

(3309, 209)

In [5]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 'points_per_game', 'total_points']]
df

,name,team_name,element_type,now_cost,points_per_game,total_points
0,Aaron Cresswell,West Ham,2,42,2.0,2
1,Aaron Hickey,Brentford,2,45,1.9,17
2,Aaron Ramsdale,Arsenal,1,50,2.8,11
3,Aaron Ramsey,Burnley,3,50,1.4,7
4,Aaron Wan-Bissaka,Manchester Utd,2,44,3.8,23
...,...,...,...,...,...,...
458,Yoane Wissa,Brentford,4,59,4.2,46
459,Youri Tielemans,Aston Villa,3,57,1.3,14
460,Youssef Ramalho Chermiti,Everton,4,49,0.8,4
461,Yves Bissouma,Tottenham,3,50,1.5,15


# Analysis

In [6]:
(projections[projections.gameweek==(latest_gameweek+1)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

,expected_points
name,
Mohamed Salah,6.531279
Bukayo Saka,6.187162
Bruno Borges Fernandes,5.914618
Heung-Min Son,5.865967
Marcus Rashford,5.809860
Ollie Watkins,5.648586
Kaoru Mitoma,5.469927
Pascal Groß,5.409401
Martin Ødegaard,5.275473


In [15]:
np.arange(latest_gameweek+1, latest_gameweek+11)

array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21])

In [19]:
(projections[(projections.gameweek.isin(np.arange(latest_gameweek+1, latest_gameweek+11))) & (projections.element_type==3)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

,expected_points
name,
Mohamed Salah,64.410403
Bukayo Saka,58.457388
Heung-Min Son,56.632934
Jarrod Bowen,49.709645
James Maddison,48.928917
Martin Ødegaard,48.858038
Bryan Mbeumo,48.793696
Raheem Sterling,48.439728
Bruno Borges Fernandes,46.209908


In [8]:
projections[projections['name'].str.contains('Moussa')].name.unique()

array(['Moussa Diaby', 'Moussa Niakhaté'], dtype=object)

In [9]:
fpl_df.loc[fpl_df['name'].str.contains('Areola'), ['name', 'gameweek', 'gameweek_minutes', 'gameweek_xPoints', ]]

,name,gameweek,gameweek_minutes,gameweek_xPoints
19098,Alphonse Areola,1,90.0,3.731472
19391,Alphonse Areola,2,90.0,5.294112
19682,Alphonse Areola,3,90.0,6.246717
19990,Alphonse Areola,4,90.0,2.578974
20293,Alphonse Areola,5,90.0,5.473296
20588,Alphonse Areola,6,90.0,2.421760
20897,Alphonse Areola,7,90.0,3.247586
21221,Alphonse Areola,8,90.0,1.948984
21524,Alphonse Areola,9,90.0,3.038878
21826,Alphonse Areola,10,90.0,4.627438


In [21]:
my_gameweek = latest_gameweek+1

my_starting_11_names = ['Areola', 'Porro', 'Cash', 'Trippier', 'Saka', 'Mitoma','Bowen', 'Son', 'Salah', 
                  'Watkins', 'Julián Álvarez']
my_subs_names = ['Turner','Tsimikas', 'Taylor', 'Archer',]

my_starting_11 = pd.DataFrame()
for my_name in my_starting_11_names:
    my_starting_11 = pd.concat([
                    my_starting_11, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

my_subs = pd.DataFrame()
for my_name in my_subs_names:
    my_subs = pd.concat([
                    my_subs, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

expected_points = my_starting_11['expected_points'].sum() + my_starting_11['expected_points'].max()
display(f'Expected points: {expected_points}')

display(my_starting_11[['name', 'element_type', 'home', 'expected_points']])
display(my_starting_11.shape[0])

display(my_subs[['name', 'element_type', 'home', 'expected_points']])
display(my_subs.shape[0])

'Expected points: 62.78213289094032'

,name,element_type,home,expected_points
209,Alphonse Areola,1,1,3.856278
3624,Pedro Porro,2,0,4.018675
3061,Matty Cash,2,1,4.245670
2520,Kieran Trippier,2,0,5.231461
685,Bukayo Saka,3,1,6.187162
2463,Kaoru Mitoma,3,1,5.469927
2019,Jarrod Bowen,3,1,5.132460
1620,Heung-Min Son,3,0,5.865967
3208,Mohamed Salah,3,1,6.531279
3525,Ollie Watkins,4,1,5.648586


11

,name,element_type,home,expected_points
3051,Matt Turner,1,0,2.751792
2540,Konstantinos Tsimikas,2,1,2.708700
805,Charlie Taylor,2,0,2.131721
735,Cameron Archer,4,0,2.168302


4

In [11]:
(projections[(projections.gameweek==(latest_gameweek+1))]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['now_cost', 'expected_points']]

,now_cost,expected_points
name,,
Mohamed Salah,129,6.531279
Bukayo Saka,86,6.187162
Bruno Borges Fernandes,83,5.914618
Heung-Min Son,97,5.865967
Marcus Rashford,87,5.809860
Ollie Watkins,84,5.648586
Kaoru Mitoma,65,5.469927
Pascal Groß,63,5.409401
Martin Ødegaard,85,5.275473


In [14]:
players = ['Bryan Mbeumo', 'Kaoru Mitoma']
colors = ['red', 'white']
#players = ['Heung-Min Son', 'Bruno Borges Fernandes']
#colors = ['white', 'red']

fig = go.Figure()
for count, player in enumerate(players):

    my_projections = projections[projections['name']==player].sort_values(by='gameweek')

    fig.add_trace(
        go.Scatter(
            x=np.unique(my_projections['gameweek']),
            y=my_projections.groupby('gameweek').sum()['expected_points'],#.cumsum(),
            mode="markers+lines",
            marker=dict(color=colors[count]),
            name=player + ' - projected',
            #hovertext=('Opponent: ' + my_projections['opp_team'] + 
            #           ', was_home: ' + my_projections['was_home'].astype(str)),
            showlegend=True,
            ),
    )

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek",
    yaxis_title='projected_points',
    #showlegend=True
)

In [ ]:
for player in players:
    my_projections = projections[projections['name']==player].sort_values(by='gameweek')
    print(player)
    display(my_projections[['opponent_team','home','gameweek', 'expected_points']].set_index('gameweek'))